In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pymorphy2

In [2]:
df = pd.read_csv("3_full_toxicity_output.csv")

In [3]:
print(len(df))
df.head()

90779


,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original
0,на свидания обычно более вежливым тоном пригл...,"давно пора выебать всех вас в жопу, пидарасов",False,True,0.000351,0.997660,На свидания обычно более вежливым тоном пригл...,"Давно пора выебать всех вас в жопу, пидарасов"
1,всем привет. вчера в b увидел фото с ру брони...,"хочу лизать ей пизду и ебать анус, а лучше еб...",False,True,0.000876,0.997648,Всем привет. Вчера в b увидел фото с Ру Брони...,"Хочу лизать ей пизду и ебать анус, а лучше еб..."
2,ты внатуре керамбит ведь тебя легко воткнуть,я твою маму прилюдно выебал в жопу достал хуй(...,True,True,0.956799,0.997591,ты внатуре керамбит ведь тебя легко воткнуть,я твою маму прилюдно выебал в жопу достал хуй(...
3,"пиздатая милфа была, ебать бы и ебать ее, но ...","пиздатая милфа была, ебать бы и ебать ее тепе...",True,True,0.996709,0.997521,"Пиздатая милфа была, ебать бы и ебать ее, но ...","Пиздатая милфа была, ебать бы и ебать ее Тепе..."
4,она не гулиха кого ты наебываешь норм тянучка,да похуй. я бы ее до смерти ебал это пиздец.,True,True,0.978831,0.997506,она не гулиха кого ты наебываешь норм тянучка,Да похуй. Я бы ее до смерти ебал это пиздец.


In [4]:
lemmatizer = pymorphy2.MorphAnalyzer()

df["Реплика 2 lemma"] = df["Реплика 2 toxicity"].apply(lambda text: [lemmatizer.parse(word)[0].normal_form for word in text.split()])



### Если есть лемматизация, то можно сразу загрузить:

In [5]:
df.to_csv("replika2_lemma.csv")

In [6]:
df.head()

,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma
0,на свидания обычно более вежливым тоном пригл...,"давно пора выебать всех вас в жопу, пидарасов",False,True,0.000351,0.997660,На свидания обычно более вежливым тоном пригл...,"Давно пора выебать всех вас в жопу, пидарасов","[давно, пора, выебать, весь, вы, в, жопу,, пид..."
1,всем привет. вчера в b увидел фото с ру брони...,"хочу лизать ей пизду и ебать анус, а лучше еб...",False,True,0.000876,0.997648,Всем привет. Вчера в b увидел фото с Ру Брони...,"Хочу лизать ей пизду и ебать анус, а лучше еб...","[хотеть, лизать, она, пизда, и, ебать, анус,, ..."
2,ты внатуре керамбит ведь тебя легко воткнуть,я твою маму прилюдно выебал в жопу достал хуй(...,True,True,0.956799,0.997591,ты внатуре керамбит ведь тебя легко воткнуть,я твою маму прилюдно выебал в жопу достал хуй(...,"[я, твой, мама, прилюдно, выебал, в, жопа, дос..."
3,"пиздатая милфа была, ебать бы и ебать ее, но ...","пиздатая милфа была, ебать бы и ебать ее тепе...",True,True,0.996709,0.997521,"Пиздатая милфа была, ебать бы и ебать ее, но ...","Пиздатая милфа была, ебать бы и ебать ее Тепе...","[пиздатай, милф, была,, ебать, бы, и, ебать, о..."
4,она не гулиха кого ты наебываешь норм тянучка,да похуй. я бы ее до смерти ебал это пиздец.,True,True,0.978831,0.997506,она не гулиха кого ты наебываешь норм тянучка,Да похуй. Я бы ее до смерти ебал это пиздец.,"[да, похуй., я, бы, она, до, смерть, ебал, это..."


In [7]:
df = pd.read_csv("replika2_lemma.csv")

### Функция, находящая ключевые слова в (лемматизированных) словах Реплики 2:

In [8]:
def get_matched_rows(df, keywords: list) -> list:
    matched = []
    for ind, (_, row) in tqdm(enumerate(df.iterrows())):
#         print(type(row))
#         print(list(row))
#         break
#     for ind, tokens in tqdm(enumerate(df["Реплика 2 lemma"].values)):
        match_words = []
        toxicity_split = row["Реплика 2 toxicity"].split(" ")
        eval_keys = eval(row["Реплика 2 lemma"])
        
        for word in keywords:
            if word in eval_keys:
                match_words.append(word)
            if word in toxicity_split and word not in match_words:
                match_words.append(word)
        
        if len(match_words) > 0:
            matched.append(list(row) + [", ".join(match_words)])
    return matched

# Женщины

In [9]:
women_moments = ['потаскуха', 'шмонька', 'блядво', 'шлюха', 'лярва', 'дебилоидка', 'блядь', 'спермоприемница', 'долбоебина', 
                 'хуесоска', 'дура', 'целка', 'фемиблядь', 'шалашовка', 'беременяшка', 'лакшовка', 'чмошница', 'блядунья', 
                 'фемоблядь', 'лахудра', 'блядва', 'алюра', 'хуеблядь', 'блядеха', 'блядище', 'давалка', 'яжмать', 'долбоебка', 
                 'шмара', 'педиковна', 'путана', 'сука', 'страхоебище', 'плешница','мымра', 'бабища', 'шалава', 'пиздушка', 
                 'разведенка', 'подзаборница', 'бабье', 'стерва', 'идиотка','блядистка', 'пиздаболка', 'беременяжка', 'курва', 
                 'блядища', 'блядюга', 'уродина', 'проститутка','баруха', 'рсп', 'дурочка', 'блядюра', 'пидораска']

In [10]:
matched_rows = get_matched_rows(df, women_moments)

90779it [00:25, 3542.75it/s]


In [11]:
women_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [12]:
women_df["target_group"] = ["women"] * len(women_df)
women_df_one = women_df.copy()

In [13]:
print(len(women_df))
women_df.head()

2388


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,6,нахуя свинозагонные русофильские пидорюндели ...,да этот загон ебаный надо удалять уже почему ...,True,True,0.994447,0.997482,Нахуя свинозагонные русофильские пидорюндели ...,Да этот загон ебаный надо удалять уже почему ...,"['да', 'этот', 'загон', 'ебаный', 'надо', 'уда...",сука,women
1,7,байден подписал законопроект об увеличении в ...,я маму этого хуяйдена ебал пусть в лицо скажет...,False,True,0.000438,0.997479,Байден подписал законопроект об увеличении в ...,я маму этого хуяйдена ебал ПУСТЬ В ЛИЦО СКАЖЕТ...,"['я', 'мама', 'это', 'хуяйден', 'ебал', 'пусть...",сука,women
2,20,долбаебы оп сказал про гипоталамус и как задей...,"гипохуямус, блять, нихуя не понятно... я хочу ...",True,True,0.990614,0.997426,долбаебы ОП сказал про гипоталамус и как задей...,"гипохуямус, блять, нихуя не понятно... Я хочу ...","['гипохуямус,', 'блять,', 'нихуй', 'не', 'поня...",сука,women
3,22,человека все устраивает в жизни и он всем дово...,у тебя когда папа видет хуй надроченный у него...,False,True,0.000719,0.997416,человека все устраивает в жизни и он всем дово...,У ТЕБЯ КОГДА ПАПА ВИДЕТ ХУЙ НАДРОЧЕННЫЙ У НЕГО...,"['у', 'ты', 'когда', 'папа', 'видти', 'хуй', '...",сука,women
4,26,итт я буду восстанавливать сей гениальный опус...,еб твою мать блядь иди отсюда нахуй блядь!,False,True,0.001949,0.997409,ИТТ я буду восстанавливать сей гениальный опус...,Ёб твою мать блядь иди отсюда нахуй блядь!,"['еб', 'твой', 'мать', 'блядь', 'идти', 'отсюд...",блядь,women


In [14]:
women_df.to_csv("women_matched.csv")

### Женщины (многозначные слова)

In [15]:
women_multiple = ['сельдь', 'гадина', 'подстилка', 'пизда', 'сука', 'мохнатка', 'блядь', 'многостаночница', 'шкура', 'блондинка',
                  'бабье']

In [16]:
matched_rows = get_matched_rows(df, women_multiple)

90779it [00:20, 4360.41it/s]


In [17]:
women_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [18]:
women_df["target_group"] = ["women"] * len(women_df)

In [19]:
print(len(women_df))
women_df.head()

2432


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,1,всем привет. вчера в b увидел фото с ру брони...,"хочу лизать ей пизду и ебать анус, а лучше еб...",False,True,0.000876,0.997648,Всем привет. Вчера в b увидел фото с Ру Брони...,"Хочу лизать ей пизду и ебать анус, а лучше еб...","['хотеть', 'лизать', 'она', 'пизда', 'и', 'еба...",пизда,women
1,6,нахуя свинозагонные русофильские пидорюндели ...,да этот загон ебаный надо удалять уже почему ...,True,True,0.994447,0.997482,Нахуя свинозагонные русофильские пидорюндели ...,Да этот загон ебаный надо удалять уже почему ...,"['да', 'этот', 'загон', 'ебаный', 'надо', 'уда...",сука,women
2,7,байден подписал законопроект об увеличении в ...,я маму этого хуяйдена ебал пусть в лицо скажет...,False,True,0.000438,0.997479,Байден подписал законопроект об увеличении в ...,я маму этого хуяйдена ебал ПУСТЬ В ЛИЦО СКАЖЕТ...,"['я', 'мама', 'это', 'хуяйден', 'ебал', 'пусть...",сука,women
3,17,сын тупой бляди я воткну копье в горло твоей б...,слушай я в пизду твоей мертвой матери сено зас...,True,True,0.995986,0.997452,сын тупой бляди я воткну копьё в горло твоей б...,слушай я в пизду твоей мёртвой матери сено зас...,"['слушай', 'я', 'в', 'пизда', 'твой', 'мёртвый...",пизда,women
4,19,"я устал жить, устал дышать, устал двигаться, я...","тоже рот все это говно ебал, в пизду эту жизн...",False,True,0.000416,0.997443,"Я устал жить, устал дышать, устал двигаться, я...","Тоже рот все это говно ебал, в пизду эту жизн...","['тоже', 'рота', 'весь', 'это', 'говно', 'ебал...",пизда,women


In [20]:
women_df.to_csv("women_multiple_matched.csv")

# ЛГБТ

In [21]:
lgbt_words = ['педерастина', 'хуесос', 'гомик', 'хуеглот', 'питух', 'педераст', 'полугомосек', 'лезбуха', 'трансуха', 'мужеложец', 
              'педермот', 'лесба', 'пидор', ' педермот', 'полупидр', 'лесбуха', 'жополюб', 'хуеблядь', 'библядь', 'пидерастина', 
              'глиномес', 'лезба', 'педиковна', 'пидорок', 'гузноеб', 'петухоеб', 'полупидор', 'жопошник', 'гейгерл', 
              'пидарас-долбоеб', 'жопник', 'трансошлюха', 'транс', 'пидрила', 'пидрилла', 'педрила', 'педрилла', 'пидорас', 
              'хуеман', 'гачеблядь', 'блядьмо', 'мужиковед', 'пидораска']
                

In [22]:
matched_rows = get_matched_rows(df, lgbt_words)

90779it [00:18, 4822.80it/s]


In [23]:
lgbt_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [24]:
lgbt_df["target_group"] = ["lgbt"] * len(lgbt_df)
lgbt_df_one = lgbt_df.copy()

In [25]:
print(len(lgbt_df))
lgbt_df.head()

998


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,12,я думал на земле нет таких людей которые смогл...,я хуесос сосу хуй за иди нахуй даун иди нахуй ...,False,True,0.001034,0.997463,я думал на земле нет таких людей которые смогл...,Я ХУЕСОС СОСУ ХУЙ ЗА 20 ИДИ НАХУЙ ДАУН ИДИ НАХ...,"['я', 'хуесос', 'сосать', 'хуй', 'за', 'идти',...",хуесос,lgbt
1,21,правльо сборнубклассическая бомжерычка в хуй,да вы заебали бляии ебаные я с телефона пишу я...,True,True,0.994193,0.997423,правльо сборнубклассическая бомжерычка в хуй,ДА ВЫ ЗАЕБАЛИ БЛЯИИ ЕБАНЫЕ Я С ТЕЛЕФОНА ПИШУ Я...,"['да', 'вы', 'заебали', 'бляие', 'ебаный', 'я'...",пидрила,lgbt
2,22,человека все устраивает в жизни и он всем дово...,у тебя когда папа видет хуй надроченный у него...,False,True,0.000719,0.997416,человека все устраивает в жизни и он всем дово...,У ТЕБЯ КОГДА ПАПА ВИДЕТ ХУЙ НАДРОЧЕННЫЙ У НЕГО...,"['у', 'ты', 'когда', 'папа', 'видти', 'хуй', '...",гомик,lgbt
3,30,зоофилов на той стороне на мой взгляд фурриеб...,"блядь, говна кусок, ты нахуй сюда это дерьмо,...",False,True,0.055261,0.997394,Зоофилов на той стороне На мой взгляд фурриеб...,"Блядь, говна кусок, ты нахуй сюда это дерьмо,...","['блядь,', 'говно', 'кусок,', 'ты', 'нахуй', '...",пидорас,lgbt
4,36,почему такой злой? тебя датчанин в попу ебал?,"тебя в жопу ебут по кд, пидрила обосранная. т...",True,True,0.995928,0.997383,Почему такой злой? тебя датчанин в попу ебал?,"Тебя в жопу ебут по КД, пидрила обосранная. Т...","['ты', 'в', 'жопа', 'ебут', 'по', 'кд,', 'пидр...",пидрила,lgbt


In [26]:
lgbt_df.to_csv("lgbt_matched.csv")

### ЛГБТ (многозначные слова)

In [27]:
lgbt_multiple = ['заднеприводный', 'транс', 'ковырялка', 'дырявый', 'петух', 'трап', 'содомит', 'трансформер']

In [28]:
matched_rows = get_matched_rows(df, lgbt_multiple)

90779it [00:18, 5031.96it/s]


In [29]:
lgbt_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [30]:
lgbt_df["target_group"] = ["lgbt"] * len(lgbt_df)

In [31]:
print(len(lgbt_df))
lgbt_df.head()

333


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,172,ты пока писал блядина сто хуев успел насосать ...,обвафленный хуеплет и сын блядушника и попугая...,True,True,0.997342,0.997235,ты пока писал блядина сто хуев успел насосать ...,обвафленный хуеплет и сын блядушника и попугая...,"['обвафленный', 'хуепгод', 'и', 'сын', 'блядуш...",дырявый,lgbt
1,218,то есть шлюхи когда делают миниет с праглотом...,"ты педераст. пасаны, пизди пидара! петухи взб...",True,True,0.981134,0.997201,То есть шлюхи когда делают миниет с праглотом...,"Ты педераст. Пасаны, пизди пидара! Петухи взб...","['ты', 'педераст.', 'пасаны,', 'пиздить', 'пид...",петух,lgbt
2,223,не пиздите на пингвина я вам башню снесу нахуу...,ебанное ничтожное чмо сидящее у мамы на шее (к...,True,True,0.991934,0.997197,НЕ ПИЗДИТЕ НА ПИНГВИНА Я ВАМ БАШНЮ СНЕСУ НАХУУ...,ЕБАННОЕ НИЧТОЖНОЕ ЧМО СИДЯЩЕЕ У МАМЫ НА ШЕЕ (К...,"['ебанный', 'ничтожный', 'чмо', 'сидеть', 'у',...",трап,lgbt
3,429,квартирник провел и кто-то короткий ролик сли...,"бля, петух ебаный. зашкварен и задеанонен. на...",False,True,0.000437,0.997091,Квартирник провёл и кто-то короткий ролик сли...,"Бля, петух ебаный. Зашкварен и задеанонен. На...","['бля,', 'петух', 'ебаный.', 'зашкварить', 'и'...",петух,lgbt
4,442,"и что, ни одного предложения об улучшении секс...",вы чо блядь твари совсем охуели уже трапов сюд...,True,True,0.981827,0.997085,"И что, ни одного предложения об улучшении секс...",ВЫ ЧО БЛЯДЬ ТВАРИ СОВСЕМ ОХУЕЛИ УЖЕ ТРАПОВ СЮД...,"['вы', 'чо', 'блядь', 'тварь', 'совсем', 'охуе...",трап,lgbt


In [32]:
lgbt_df.to_csv("lgbt_multiple_matched.csv")

# Мигранты / по цвету кожи

In [33]:
migrant_words = ['хачамбрий', 'хачешарик', 'хачсан', 'джамшут', 'хачара', 'нигарита', 'черножопый', 'нигавный', 'желтопузый', 
                 'хачкор', 'хачуга', 'пиздоглазый', 'белозадый', 'неггереныш', 'хачка', 'асланбек-мамкоеб', 'хачлом', 'нигра', 
                 'хачик', 'абу', 'нигрилла', 'ниггерша', 'чурбек', 'бекоеб', 'хачтак', 'хачуа', 'скотоебина', 'хачила', 
                 'чероножопый', 'хачиха', 'хач', 'черномазый', 'нигредо', 'черныш', 'чучмек', 'нигга', 'нигретенок', 'хачина', 
                 'нерусь', 'хачетяночка', 'хачинк', 'беломазый', 'нига', 'хачтака', 'хачкель', 'ниггер', 'узкоглазый', 'нигерша', 
                 'хачевата', 'нигрила', 'чурка', 'азиатоблядь', 'хачебеска', 'нигер']

In [34]:
matched_rows = get_matched_rows(df, migrant_words)

90779it [00:20, 4340.27it/s]


In [35]:
migrant_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [36]:
migrant_df["target_group"] = ["migrants"] * len(migrant_df)
migrant_df_one = migrant_df.copy()

In [37]:
print(len(migrant_df))
migrant_df.head()

531


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,6,нахуя свинозагонные русофильские пидорюндели ...,да этот загон ебаный надо удалять уже почему ...,True,True,0.994447,0.997482,Нахуя свинозагонные русофильские пидорюндели ...,Да этот загон ебаный надо удалять уже почему ...,"['да', 'этот', 'загон', 'ебаный', 'надо', 'уда...",абу,migrants
1,63,обновляю страницу раз в пару дней и долбаебы о...,та сьеби отсюда далбаеб парализованный здесь с...,True,True,0.987258,0.997341,обновляю страницу раз в пару дней И долбаебы о...,та сьеби отсюда далбаеб парализованный здесь с...,"['тот', 'сьести', 'отсюда', 'далбаести', 'пара...",чурка,migrants
2,138,планирую все-таки перестать писать про дашу. я...,ее там ебали жестко в рот и в жопу особенно то...,False,True,0.000530,0.997266,Планирую все-таки перестать писать про Дашу. Я...,Ее там ебали жёстко в рот и в жопу особенно то...,"['она', 'там', 'ебали', 'жёстко', 'в', 'рота',...",чурка,migrants
3,203,арендуем зал ставлю против тебя шарлея или дже...,маму свою выеби за такое уебищное имя наиль бл...,True,True,0.926096,0.997214,Арендуем зал ставлю против тебя шарлея или дже...,маму свою выеби за такое уебищное имя НАИЛЬ БЛ...,"['мама', 'свой', 'выести', 'за', 'такой', 'уеб...",чурка,migrants
4,220,"уебок я миллион фоток через ремини прогнал, он...",тупая пиздоглазая скатина на твоем скрине тоже...,True,True,0.994955,0.997200,"Уёбок я миллион фоток через ремини прогнал, он...",тупая пиздоглазая скатина на твоем скрине тоже...,"['тупой', 'пиздоглазый', 'скатин', 'на', 'твой...",пиздоглазый,migrants


In [38]:
migrant_df.to_csv("migrant_matched.csv")

### Мигранты / по цвету кожи (многозначные слова)

In [39]:
migrant_multiple = ['черныш', 'обезьяна']

In [40]:
matched_rows = get_matched_rows(df, migrant_multiple)

90779it [00:15, 5923.62it/s]


In [41]:
migrant_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [42]:
migrant_df["target_group"] = ["migrants"] * len(migrant_df)

In [43]:
print(len(migrant_df))
migrant_df.head()

99


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,193,"надеемся, что правоохранительные органы заинт...",выкинуть нахуй из рашки обезьяны заебали хуйне...,False,True,0.000286,0.997219,"Надеемся, что правоохранительные органы заинт...",выкинуть нахуй из рашки обезьяны заебали хуйне...,"['выкинуть', 'нахуй', 'из', 'рашка', 'обезьяна...",обезьяна,migrants
1,627,"прет, суи, забавная история, хотел тебя побла...",тебя добить надо. замочить. и жену твою срану...,False,True,0.009613,0.997014,"Прет, Суи, забавная история, хотел тебя побла...",Тебя добить надо. Замочить. И жену твою срану...,"['ты', 'добить', 'надо.', 'замочить.', 'и', 'ж...",обезьяна,migrants
2,5198,изображен солнцем и его называют солнцем хуенц...,да причем тут солнце долбоеб? солнце это блят...,True,True,0.994433,0.995962,изображен солнцем и его называют солнцем Хуёнц...,Да причем тут солнце долбоеб? Солнце это блят...,"['да', 'причём', 'тут', 'солнце', 'долбоеб?', ...",обезьяна,migrants
3,5566,смотря с какой передачи. если стартуешь с - п...,"обезьяна шатал макабу, разметка вдоль пизды п...",False,True,0.002736,0.995891,Смотря с какой передачи. Если стартуешь с 38 ...,"Обезьяна шатал макабу, разметка вдоль пизды п...","['обезьяна', 'шатать', 'макабу,', 'разметка', ...",обезьяна,migrants
4,6617,"мне похуй на твоего олаха, контакты оставь, ом...",обратись к обезьяне этого гадюшника жидовског...,True,True,0.991487,0.995700,"мне похуй на твоего олаха, контакты оставь, ом...",Обратись к обезьяне этого гадюшника жидовског...,"['обратиться', 'к', 'обезьяна', 'это', 'гадюшн...",обезьяна,migrants


In [44]:
migrant_df.to_csv("migrant_multiple_matched.csv")

# Мужчины

In [45]:
man_words = ['далбаеб', 'еблан', 'сыкун', 'жидяра', 'очкун', 'уродец', 'пиздонович', 'отец-мочехлеб', 'алкаш', 'мудила', 'шлюхоеб',
             'аутист', 'мужло', 'говнюк', 'жиденыш', 'блядун', 'мудаеб', 'ебанат', 'ебасос', 'мерзавец', 'овердолбоеб', 'объебыш',
             'хуерик', 'страхоебище', 'чмошник', 'утырок', 'мудак', 'пидрила', 'пидорас', 'подкуколдыш', 'распиздяй', 'хуесос',
             'уебок', 'пиздюлевич', 'ублюдок', 'куколд', 'пердун', 'засерун', 'хуйло', 'пиздализ', 'хуеблядь', 'балбес', 'обмудок',
             'пиздабол', 'долбоеб', 'спермотоксикозник', 'альфач', 'хуеплет', 'подпездыш', 'пиздолиз', 'родитель-долбоеб',
             'анимедолбоеб', 'ебанатик', 'оладух', 'дебилоид', 'ссыкло', 'батька-долбоеб', 'дрищ', 'разведеныш', 'кретин',
             'объебщик', 'чмудак', 'пидор', 'мудотяп', 'мудачина', 'потаскун', 'кривохуй', 'дуроеб', 'пидорок', 'хуило', 'говноеб',
             'ебырь', 'хер', 'хрыч', 'мамкоеб', 'сcыкун', 'омежка-всратыш', 'придурок', 'кукколд', 'блядович', 'убердалбаеб',
             'спермобак', 'ебарь', 'хуемразь', 'долбаеб', 'урод', 'дебил', 'пиздокрыл', 'блядолиз', 'подонок', 'шизодолбоеб', 'жид',
             'идиот', 'гандонович', 'мудень', 'педрила', 'педрилла', 'пидрила', 'пидрилла']

In [46]:
matched_rows = get_matched_rows(df, man_words)

90779it [00:21, 4155.52it/s]


In [47]:
man_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [48]:
man_df["target_group"] = ["man"] * len(man_df)
man_df_one = man_df.copy()

In [49]:
print(len(man_df))
man_df.head()

4572


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,10,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,True,True,0.979576,0.997473,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,"['ты', 'хуепгод', 'ебаный', 'с', 'еблискать', ...",хуеплет,man
1,12,я думал на земле нет таких людей которые смогл...,я хуесос сосу хуй за иди нахуй даун иди нахуй ...,False,True,0.001034,0.997463,я думал на земле нет таких людей которые смогл...,Я ХУЕСОС СОСУ ХУЙ ЗА 20 ИДИ НАХУЙ ДАУН ИДИ НАХ...,"['я', 'хуесос', 'сосать', 'хуй', 'за', 'идти',...",хуесос,man
2,21,правльо сборнубклассическая бомжерычка в хуй,да вы заебали бляии ебаные я с телефона пишу я...,True,True,0.994193,0.997423,правльо сборнубклассическая бомжерычка в хуй,ДА ВЫ ЗАЕБАЛИ БЛЯИИ ЕБАНЫЕ Я С ТЕЛЕФОНА ПИШУ Я...,"['да', 'вы', 'заебали', 'бляие', 'ебаный', 'я'...",пидрила,man
3,27,"отлично, теперь пойди черниговским мiцным убей...",ты че мелишь клякса ебучая ебанат это ты делал...,False,True,0.423887,0.997406,"Отлично, теперь пойди Черниговским мiцным убей...",ты че мелишь клякса ебучая ебанат это ты делал...,"['ты', 'че', 'мелить', 'клякса', 'ебучаять', '...",ебанат,man
4,28,вся суть понадусеровых швайнокарасей,"- ты кого фашистом назвал, уебок ты блять? - т...",False,True,0.186888,0.997404,Вся суть понадусеровых швайнокарасей,"- Ты кого фашистом назвал, уёбок ты блять? - Т...","['-', 'ты', 'кто', 'фашист', 'назвал,', 'уебок...",уебок,man


In [50]:
man_df.to_csv("man_matched.csv")

### Мужчины (многозначные слова)

In [51]:
man_multiple = ['гандон', 'хуй', 'хер', 'хуи', 'гондон']

In [52]:
matched_rows = get_matched_rows(df, man_multiple)

90779it [00:21, 4186.38it/s]


In [53]:
man_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [54]:
man_df["target_group"] = ["man"] * len(man_df)

In [55]:
print(len(man_df))
man_df.head()

4285


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,8,"да все холодные страны можно, на глубине км у...","хуй ты мой сосал, пися недоразвитая. я твою с...",False,True,0.000389,0.997477,"Да все холодные страны можно, на глубине 5км ...","Хуй ты мой сосал, пися недоразвитая. Я твою с...","['хуй', 'ты', 'мой', 'сосал,', 'пися', 'недора...",хуй,man
1,9,"но днем солнце есть, а звезд нет, поэтому, ну...","хуй ты мой сосал, пися недоразвитая. я твою с...",False,True,0.000392,0.997477,"Но днём солнце есть, а звёзд нет, поэтому, ну...","Хуй ты мой сосал, пися недоразвитая. Я твою с...","['хуй', 'ты', 'мой', 'сосал,', 'пися', 'недора...",хуй,man
2,10,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,True,True,0.979576,0.997473,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,"['ты', 'хуепгод', 'ебаный', 'с', 'еблискать', ...",хуй,man
3,11,ты громосекус... просто твоя история это апог...,"щас бы за касарь пидарасить, сосать и дрочить...",False,True,0.009809,0.997469,Ты громосекус... Просто твоя история это апог...,"Щас бы за касарь пидарасить, сосать и дрочить...","['щас', 'бы', 'за', 'касарь', 'пидарасить,', '...",гандон,man
4,12,я думал на земле нет таких людей которые смогл...,я хуесос сосу хуй за иди нахуй даун иди нахуй ...,False,True,0.001034,0.997463,я думал на земле нет таких людей которые смогл...,Я ХУЕСОС СОСУ ХУЙ ЗА 20 ИДИ НАХУЙ ДАУН ИДИ НАХ...,"['я', 'хуесос', 'сосать', 'хуй', 'за', 'идти',...",хуй,man


In [56]:
man_df.to_csv("man_multiple_matched.csv")

# По коренному происхождению

In [57]:
born_place_words = ['жидоблядь', 'хохленыш', 'ослоеб', 'кацап', 'жидяра', 'чуркобес', 'маскаль', 'маскваблядь', 'свиноеб', 
                    'шведоблядь', 'жиденыш', 'макаронник', 'хачик', 'пендос', 'козлоеб', 'овцееб', 'скотоебина', 'москаль', 
                    'хачиха', 'скандинавоблядь', 'хач', 'хохлоблядь', 'хохложопый', 'руснявыш', 'конееб', 'русня', 'хачетяночка', 
                    'саложопый', 'раски', 'полукровка', 'рассеянезадый', 'пиндос', 'чурка', 'москваблядь', 'азиатоблядь', 'хохол', 
                    'жид', 'чуркабес', 'азер']

In [58]:
matched_rows = get_matched_rows(df, born_place_words)

90779it [00:22, 4030.61it/s]


In [59]:
born_place_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [60]:
born_place_df["target_group"] = ["born_place"] * len(born_place_df)
born_place_df_one = born_place_df.copy()

In [61]:
print(len(born_place_df))
born_place_df.head()

1328


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,49,бля зоны роста это когда хрящи костей заполня...,"блять уебок ебучий, я тебе сказал нахуй я гото...",True,True,0.913245,0.997365,Бля зоны роста это когда хрящи костей заполня...,"блять уебок ебучий, я тебе сказал нахуй я гото...","['блять', 'уебок', 'ебучий,', 'я', 'ты', 'сказ...",хохол,born_place
1,63,обновляю страницу раз в пару дней и долбаебы о...,та сьеби отсюда далбаеб парализованный здесь с...,True,True,0.987258,0.997341,обновляю страницу раз в пару дней И долбаебы о...,та сьеби отсюда далбаеб парализованный здесь с...,"['тот', 'сьести', 'отсюда', 'далбаести', 'пара...",чурка,born_place
2,138,планирую все-таки перестать писать про дашу. я...,ее там ебали жестко в рот и в жопу особенно то...,False,True,0.000530,0.997266,Планирую все-таки перестать писать про Дашу. Я...,Ее там ебали жёстко в рот и в жопу особенно то...,"['она', 'там', 'ебали', 'жёстко', 'в', 'рота',...",чурка,born_place
3,150,"путин просто поставил всю страну на колени, пр...",моча говно нассал опустил в жопу выебал какие ...,False,True,0.113696,0.997252,"путин просто поставил всю страну на колени, пр...",Моча говно нассал опустил в жопу выебал какие ...,"['моча', 'говно', 'нассать', 'опустить', 'в', ...",хохол,born_place
4,203,арендуем зал ставлю против тебя шарлея или дже...,маму свою выеби за такое уебищное имя наиль бл...,True,True,0.926096,0.997214,Арендуем зал ставлю против тебя шарлея или дже...,маму свою выеби за такое уебищное имя НАИЛЬ БЛ...,"['мама', 'свой', 'выести', 'за', 'такой', 'уеб...",чурка,born_place


In [62]:
born_place_df.to_csv("born_place_matched.csv")

### По коренному происхождению (многозначные слова)

In [63]:
born_place_multiple = ['кокос', 'макака', 'орео', 'урюк', 'укроп', 'чебурек', 'хохол']

In [64]:
matched_rows = get_matched_rows(df, born_place_multiple)

90779it [00:21, 4241.12it/s]


In [65]:
born_place_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [66]:
born_place_df["target_group"] = ["born_place"] * len(born_place_df)

In [67]:
print(len(born_place_df))
born_place_df.head()

613


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,49,бля зоны роста это когда хрящи костей заполня...,"блять уебок ебучий, я тебе сказал нахуй я гото...",True,True,0.913245,0.997365,Бля зоны роста это когда хрящи костей заполня...,"блять уебок ебучий, я тебе сказал нахуй я гото...","['блять', 'уебок', 'ебучий,', 'я', 'ты', 'сказ...",хохол,born_place
1,150,"путин просто поставил всю страну на колени, пр...",моча говно нассал опустил в жопу выебал какие ...,False,True,0.113696,0.997252,"путин просто поставил всю страну на колени, пр...",Моча говно нассал опустил в жопу выебал какие ...,"['моча', 'говно', 'нассать', 'опустить', 'в', ...",хохол,born_place
2,280,в ермолино московской области поймали тигренка...,а тем временем хохлы в жопу ебутся надев на го...,False,True,0.000680,0.997162,В Ермолино Московской области поймали тигренка...,А тем временем хохлы в жопу ебутся надев на го...,"['а', 'тем', 'время', 'хохол', 'в', 'жопа', 'е...",хохол,born_place
3,336,"пидорашка чужими тик-токами превозмагает, пок...",тупорылый хуесосник говноед хохол картиночкам...,True,True,0.996995,0.997136,"Пидорашка чужими тик-токами превозмагает, пок...",Тупорылый хуесосник говноед хохол картиночкам...,"['тупорылый', 'хуесосникнуть', 'говноед', 'хох...",хохол,born_place
4,356,бараш пытается сжечь хохла живьем пофиксил,барин пытается сжечь хохла живьем заебали.,True,True,0.989871,0.997129,Бараш пытается сжечь хохла живьём Пофиксил,Барин пытается сжечь хохла живьем Заебали.,"['барин', 'пытаться', 'сжечь', 'хохол', 'живьё...",хохол,born_place


In [68]:
born_place_df.to_csv("born_place_multiple.csv")

# Дети/подростки

In [69]:
child_words = ['выродок', 'припездыш', 'чмыреныш', 'школопендрик', 'школота', 'сучонышсученыш', 'школоеб', 'хохленыш', 
               'абортыш', 'школопендра', 'школопетушок', 'дебиленыш', 'всратыш', 'сыкун', 'сблевыш', 'выблядок', 'школоблядь', 
               'ссыкун', 'обосыш', 'неггереныш', 'ушлепок', 'школоуебок', 'школодолбоеб', 'школопитек', 'крысеныш', 'жиденыш', 
               'зумереныш', 'балбес', 'щкильник', 'обоссыш', 'школьник-долбоеб', 'шкильница', 'хуеныш', 'спермотоксикозник', 
               'обосрыш', 'школонацик', 'школодебиленок', 'приемыш', 'высрыш', 'гаденыш', 'мелкоблядь', 'нигретенок', 'школенок', 
               'пиздючка', 'школолошник', 'визглявыш', 'залупыш', 'школьнига', 'подменыш', 'гниденыш', 'хуегрыз', 'пизденыш', 
               'ебанатик', 'пиздюк', 'пездюк', 'школолошка', 'припиздыш', 'спиногрыз', 'личинус', 'щкильница', 'сученыш']

In [70]:
matched_rows = get_matched_rows(df, child_words)

90779it [00:23, 3806.59it/s]


In [71]:
child_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [72]:
child_df["target_group"] = ["child"] * len(child_df)
child_df_one = child_df.copy()

In [73]:
print(len(child_df))
child_df.head()

421


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,6,нахуя свинозагонные русофильские пидорюндели ...,да этот загон ебаный надо удалять уже почему ...,True,True,0.994447,0.997482,Нахуя свинозагонные русофильские пидорюндели ...,Да этот загон ебаный надо удалять уже почему ...,"['да', 'этот', 'загон', 'ебаный', 'надо', 'уда...",выблядок,child
1,12,я думал на земле нет таких людей которые смогл...,я хуесос сосу хуй за иди нахуй даун иди нахуй ...,False,True,0.001034,0.997463,я думал на земле нет таких людей которые смогл...,Я ХУЕСОС СОСУ ХУЙ ЗА 20 ИДИ НАХУЙ ДАУН ИДИ НАХ...,"['я', 'хуесос', 'сосать', 'хуй', 'за', 'идти',...",выблядок,child
2,22,человека все устраивает в жизни и он всем дово...,у тебя когда папа видет хуй надроченный у него...,False,True,0.000719,0.997416,человека все устраивает в жизни и он всем дово...,У ТЕБЯ КОГДА ПАПА ВИДЕТ ХУЙ НАДРОЧЕННЫЙ У НЕГО...,"['у', 'ты', 'когда', 'папа', 'видти', 'хуй', '...",выблядок,child
3,31,я та шмара безногая которую адольф на столе дл...,ты уебище бездарное твоя мать сосала хуй у каж...,True,True,0.887016,0.997388,я та шмара безногая которую адольф на столе дл...,ты уебище бездарное ТВОЯ МАТЬ СОСАЛА ХУЙ У КАЖ...,"['ты', 'уебище', 'бездарный', 'твой', 'мать', ...",выродок,child
4,33,"вчера гулиху очень хорошо отъебал раз, ей очен...",даня ты заебало закнись нахуй ты заебал менгя ...,False,True,0.043827,0.997387,"Вчера гулиху очень хорошо отъебал 1 раз, ей оч...",даня ты заебало ЗАКНИСЬ нахуй ты заебал менгя ...,"['даня', 'ты', 'заебало', 'закнуться', 'нахуй'...",выблядок,child


In [74]:
child_df.to_csv("child_matched.csv")

### Дети/подростки (многозначные слова)

In [75]:
child_multiple = ['крысеныш']

In [76]:
matched_rows = get_matched_rows(df, child_multiple)

90779it [00:18, 4826.80it/s]


In [77]:
child_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [78]:
child_df["target_group"] = ["child"] * len(child_df)

In [79]:
print(len(child_df))
child_df.head()

1


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,40446,какой же ты все-таки самовлюбленный дегенерат...,"самовлюбленный дегенерата это лучше или хуже,...",True,True,0.992399,0.982322,Какой же ты все-таки самовлюбленный дегенерат...,"самовлюбленный дегенератА это лучше или хуже,...","['самовлюблённый', 'дегенерат', 'это', 'хороши...",крысеныш,child


In [80]:
child_df.to_csv("child_multiple.csv")

# Others

In [81]:
others_words = ['заморыш', 'ципсоблядь', 'гитлерблядь', 'мкадыш', 'вкалтактыш', 'жидяра', 'конеблядь', 'говненыш', 'бампблять', 
                'грозоеб', 'зароблять', 'скобкоблядь', 'успешноблядь', 'говножопый', 'лайкоблядь', 'жиденыш', 'дохлоблядь', 
                'лифтоеб', 'совкоблядь', 'совкожопый', 'анимуеб', 'рваножопый', 'мега-еб', 'ковидозаеб', 'минькоеб', 'замкадыш', 
                'метаблядь', 'хачтаковец', 'кремлебот-долбоеб', 'музлоеб', 'мразота', 'старожопый', 'рептилоблядь', 'грузоблядь', 
                'никчемыш', 'киберблядь', 'патриблядь', 'любероеб', 'шароеб', 'жопошник', 'цскалыш', 'вакциноблядь', 'пенсоблядь',
                'уебыш', 'трипоблядыш', 'детоеб', 'нациблядь', 'жирножопый', 'тазоеб', 'обнуленыш', 'лахтеныш', 'пидорас', 
                'зашуганыш', 'собакаеб', 'пидрила', 'ваксероблядь', 'спортоблядь', 'сракоеб', 'тянкаеб', 'скотобаза', 'анимеблядь', 
                'какатыш', 'слабожопый', 'теслоеб', 'отыгрыш', 'аватаркоблядь', 'ебанько', 'вахаеб', 'калоеб', 'вакциноеб', 
                'инст-блядь', 'мясоблядь', 'прулееб', 'фуриеб', 'хитрожопый', 'некроеб', 'каргоблядь', 'ножеблядь', 'жиробасина', 
                'мотоблядь', 'бомжееб', 'короноблядь', 'девяностыш', 'наркоблядь', 'подсисяныш', 'вагоблядь', 'зашугыш', 
                'обосраныш', 'религиоблядь', 'толстожопый', 'таксоблядь', 'вероблядь', 'мочехлеб', 'ведроеб', 'пидораска', 
                'зачморыш', 'бородоблядь', 'жиртрест', 'тюрьмыблядь', 'перекатаблять', 'наци-долбоеб', 'свиноблять', 'дрищеблядь', 
                'седок-долбоеб', 'пешеблядь', 'жид', 'лошара', 'нищееб', 'хуеплет', 'нищеблядь', 'забитыш', 'грузоеб', 'тороблядь',
                'спермохлеб', 'пешеблять', 'мразь', 'книгоблядь', 'инстаблядь', 'трапоеб', 'фашистоеб', 'крысоблядь', 'жирнозадый', 
                'собакоблядь', 'ногоблядь', 'обдолбыш', 'страваеб', 'витчеблядь', 'косоеб', 'китаееб', 'хач-русофоб', 'совкоеб', 
                'плохожопый', 'фронтатеб', 'таетоблядь', 'айтиблядь', 'оглоеб', 'школотрон', 'тазоблядь', 'евроблядь', 'паниеб', 
                'хачкор', 'обосраткыш', 'пидор', 'куклоеб', 'котоеб', 'кровожопый', 'дисиеб', 'кривожопый', 'пынееб', 'штырееб',
                'порашеблядь', 'социоблядь', 'мемоеб', 'велоблядь', 'землееб', 'высерыш', 'манкоблядь', 'даэдроеб', 'последыш', 
                'подлюга', 'скотоебина', 'тесло-долбаеб', 'лузер-долбаеб', 'кодексаеб', 'пидорок', 'рукожопый', 'бомпоеб', 
                'головожопый', 'жожоблядь', 'фурриеб', 'ваксоблядь', 'зубоблядь', 'наебыш', 'нищеброд', 'христоблядь', 'вероеб', 
                'жироблядь', 'автоблять', 'антиваксоблядь', 'карликоеб', 'облевыш', 'аркоеб', 'потреблядь', 'свиноблядь', 
                'шифтоблядь', 'челяба-еб', 'анимублядь', 'выпердыш', 'лахтоблядь', 'подхуйленыш', 'техноеб', 'лахтыш', 'трепло', 
                'стрипсоеб', 'терпила', 'сатурноеб', 'эплоблядь', 'вегоблядь', 'обосратыш', 'тоетаблядь', 'сволочь', 'конфоблядь', 
                'взблевыш', 'лахтоблять', 'ютубдолбоеб', 'рукажопый', 'нигрипес', 'передачблять', 'автоблядь', 'нормиблядь', 
                'дегенерат', 'проблядь', 'фуроеб', 'вахоеб', 'бердыш', 'лахтовыблядь', 'промытыш', 'торчееб', 'котоблядь', 
                'марвелоблядь', 'ротожопый', 'плоскожопый', 'залетыш', 'капиталоблядь', 'дебилявыш', 'ксеноеб', 'ковидоблядь', 
                'свиножопый', 'чмо', 'работоблядь', 'нигрол', 'впблять', 'мангаеб', 'мимодробыш', 'скотоблядь', 'ложкоблядь', 
                'афтоблядь', 'ашаноблядь', 'оборвыш', 'говногреб', 'широкожопый', 'модоеб', 'фентезиеб', 'собакоеб', 
                'электроблядь', 'либерал-долбоеб', 'хайпоблядь', 'еболдыш', 'ботоблядь', 'отбитыш', 'бомжеблядь', 'соловьеб', 
                'кризисаеб', 'пидрилла', 'педрила', 'педрилла']

In [82]:
matched_rows = get_matched_rows(df, others_words)

90779it [00:32, 2776.89it/s]


In [83]:
others_df = pd.DataFrame(matched_rows, columns=list(df.columns) + ["Выделенные слова"])

In [84]:
others_df["target_group"] = ["other"] * len(others_df)

In [85]:
print(len(others_df))
others_df.head()

2105


,Unnamed: 0,Реплика 1 toxicity,Реплика 2 toxicity,Результат 1,Результат 2,Значение 1,Значение 2,Реплика 1 original,Реплика 2 original,Реплика 2 lemma,Выделенные слова,target_group
0,10,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,True,True,0.979576,0.997473,сын шлюхи плакаты от картин уже отличить не мо...,ты хуеплет ебаный с еблищем огр мага нахуй воо...,"['ты', 'хуепгод', 'ебаный', 'с', 'еблискать', ...",хуеплет,other
1,21,правльо сборнубклассическая бомжерычка в хуй,да вы заебали бляии ебаные я с телефона пишу я...,True,True,0.994193,0.997423,правльо сборнубклассическая бомжерычка в хуй,ДА ВЫ ЗАЕБАЛИ БЛЯИИ ЕБАНЫЕ Я С ТЕЛЕФОНА ПИШУ Я...,"['да', 'вы', 'заебали', 'бляие', 'ебаный', 'я'...",пидрила,other
2,30,зоофилов на той стороне на мой взгляд фурриеб...,"блядь, говна кусок, ты нахуй сюда это дерьмо,...",False,True,0.055261,0.997394,Зоофилов на той стороне На мой взгляд фурриеб...,"Блядь, говна кусок, ты нахуй сюда это дерьмо,...","['блядь,', 'говно', 'кусок,', 'ты', 'нахуй', '...","пидорас, собакоеб",other
3,36,почему такой злой? тебя датчанин в попу ебал?,"тебя в жопу ебут по кд, пидрила обосранная. т...",True,True,0.995928,0.997383,Почему такой злой? тебя датчанин в попу ебал?,"Тебя в жопу ебут по КД, пидрила обосранная. Т...","['ты', 'в', 'жопа', 'ебут', 'по', 'кд,', 'пидр...","пидрила, дегенерат",other
4,38,проснулся - скурил чью-то зарплату,пидорас я твою башку по асфальту размажу блять,False,True,0.001608,0.997380,проснулся - скурил чью-то зарплату,пидорас я твою башку по асфальту размажу блять,"['пидорас', 'я', 'твой', 'башка', 'по', 'асфал...",пидорас,other


In [86]:
others_df.to_csv("others_words.csv")